В данном ноутбуке реализованы все необходимые шаги для SFT пайплайна, включая подготовку датасета.

# Импорты

In [1]:
from dotenv import load_dotenv
import sys
sys.path.append("./src")
import os
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

from src import custom_dataset
from src.train import train

import clearml

os.environ["WANDB_DISABLED"] = "true"

e:\cv-generation-git\train_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Задаем требуемые константы

In [1]:
DATA_DIR = "./data"

# соотношение train и eval датасетов для SFT пайплайна
TRAIN_RATIO = 0.9

MODEL_NAME = "IlyaGusev/rut5_base_sum_gazeta"

# Функции

In [2]:
def get_train_eval(df, train_ratio: float):
    """ создает train и eval датасеты указанной пропорции"""                                                  
                                                    
    num_rows = int(len(df) * train_ratio)
    
    train_df = df[:num_rows]
    eval_df = df[num_rows:]
    
    return train_df, eval_df                        

# Готовим данные для этапа SFT

Загружаем датасет, содержащий примеры поля "о себе" в резюме, написанные людьми

In [ ]:
df = pd.read_csv(os.path.join(DATA_DIR, "sft_dataset.csv"))

In [ ]:
df.drop(axis=1, columns=["Unnamed: 0"], inplace=True)

,source,target
0,Пол: Мужчина. Желаемая зарплата: от 200000.0 р...,"Мне - 60-летнему мужчине, проживающему в Москв..."
1,Пол: Мужчина. Желаемая зарплата: от 25000.0 ру...,Опытный бухгалтер с 53-летним стажем и опытом ...
2,Пол: Мужчина. Желаемая зарплата: от 150000.0 р...,Я - опытный сотрудник службы безопасности с бо...
3,Пол: Мужчина. Желаемая зарплата: от 190000.0 р...,Я опытный заместитель генерального директора с...
4,Пол: Женщина. Желаемая зарплата: от 0.0 рублей...,Я педагог с большим опытом работы в этой облас...


## Генерим финальные датасеты train и eval для запуска SFT

In [ ]:
train_df, eval_df = get_train_eval(df, TRAIN_RATIO)
train_df.shape

(1417, 2)

In [ ]:
eval_df.shape

(158, 2)

In [ ]:
train_df.to_json(f"{DATA_DIR}/train.json", orient="records", lines=True)
eval_df.to_json(f"{DATA_DIR}/eval.json", orient="records", lines=True)

# Запускаем SFT

In [3]:
task = clearml.Task.init(project_name='cv-generation', task_name='clean-only-sft-2')
training_conf_args = {
    "config_file": "t5_training_config.json",
    "checkpoint": "./models/sft_clean_only/checkpoint-140",
    "train_file": f"{DATA_DIR}/train.json",
    "val_file": f"{DATA_DIR}/eval.json",
    "output_dir": "./models/sft_clean_only_2",
    "train_sample_rate": 3,
    "val_sample_rate": 3,
    "seed": 42,
    "source_field": "source",
    "target_field": "target"
}

train(**training_conf_args)
task.close()

ClearML Task: created new task id=7bf08c37521c46ffbc6fd5267386b0fb
2024-06-16 05:32:58,800 - clearml.Repository Detection - WARNING - Could not read Jupyter Notebook: No module named 'nbconvert'
2024-06-16 05:32:58,818 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/4dfada0d47c54d358198351db8a35944/experiments/7bf08c37521c46ffbc6fd5267386b0fb/output/log


loading file spiece.model from cache at C:\Users\yxtag/.cache\huggingface\hub\models--IlyaGusev--rut5_base_sum_gazeta\snapshots\f09a08cae5d74c70e55da1a6ebb49f88c26f433b\spiece.model
loading file tokenizer.json from cache at C:\Users\yxtag/.cache\huggingface\hub\models--IlyaGusev--rut5_base_sum_gazeta\snapshots\f09a08cae5d74c70e55da1a6ebb49f88c26f433b\tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at C:\Users\yxtag/.cache\huggingface\hub\models--IlyaGusev--rut5_base_sum_gazeta\snapshots\f09a08cae5d74c70e55da1a6ebb49f88c26f433b\special_tokens_map.json
loading file tokenizer_config.json from cache at C:\Users\yxtag/.cache\huggingface\hub\models--IlyaGusev--rut5_base_sum_gazeta\snapshots\f09a08cae5d74c70e55da1a6ebb49f88c26f433b\tokenizer_config.json


Vocab size:  30000
PAD:  0 <pad>
BOS:  None None
EOS:  1 </s>
UNK:  2 <unk>
SEP:  None None


100%|██████████| 158/158 [00:00<00:00, 1516.30it/s]
loading configuration file config.json from cache at C:\Users\yxtag/.cache\huggingface\hub\models--IlyaGusev--rut5_base_sum_gazeta\snapshots\f09a08cae5d74c70e55da1a6ebb49f88c26f433b\config.json
Model config T5Config {
  "_name_or_path": "IlyaGusev/rut5_base_sum_gazeta",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "bos_token_id": 2,
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 2,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "max_length": 200,
  "model_type": "t5",
  "num_beams": 5,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_wor

2024-06-16 05:33:05,097 - clearml.model - INFO - Selected model id: 78adba2ead194f4a872f471f12b77b47


...

 99%|█████████▉| 288/290 [4:46:49<03:41, 110.50s/it]

{'loss': 0.9334, 'learning_rate': 4.210526315789474e-06, 'epoch': 9.8}


100%|██████████| 290/290 [4:50:22<00:00, 108.58s/it]

Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./models/sft_clean_only_2\checkpoint-260 (score: 0.9652692675590515).
There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].
100%|██████████| 290/290 [4:50:23<00:00, 60.08s/it] 
Configuration saved in ./models/sft_clean_only_2\config.json
Configuration saved in ./models/sft_clean_only_2\generation_config.json


{'train_runtime': 17426.1691, 'train_samples_per_second': 0.813, 'train_steps_per_second': 0.017, 'train_loss': 0.5068400835168773, 'epoch': 9.87}


Model weights saved in ./models/sft_clean_only_2\pytorch_model.bin
tokenizer config file saved in ./models/sft_clean_only_2\tokenizer_config.json
Special tokens file saved in ./models/sft_clean_only_2\special_tokens_map.json
